<a href="https://colab.research.google.com/github/yashk2000/BolleRenverse/blob/master/charRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [0]:
with open('percy.txt', 'r') as f:
  text = f.read()

In [49]:
text[:10000]

'ONE \n\nI GO CRUISING WITH \nEXPLOSIVES \n\nThe end of the world started when a pegasus landed on the hood of my car. \n\nUp until then, I was having a great afternoon. Technically I wasn’t supposed to be driving \nbecause I wouldn’t turn sixteen for another week, but my mom and my stepdad, Paul, took my \nfriend Rachel and me to this private stretch of beach on the South Shore, and Paul let us borrow \nhis Prius for a short spin. \n\nNow, 1 know you\'re thinking, Wow, that was really irresponsible of him, blah, blah, blah, but Paul \nknows me pretty well. He\'s seen me slice up demons and leap out of exploding school buildings, so he \nprobably figured taking a car a few hundred yards wasn\'t exactly the most dangerous thing I\'d ever done. \n\nAnyway, Rachel and I were driving along. It was a hot August day. Rachel\'s red hair was pulled back \nin a ponytail and she wore a white blouse over her swimsuit. I\'d never seen her in anything but ratty T- \nshirts and paint-splattered jean

In [0]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])

In [51]:
encoded[:100]

array([46,  0, 26, 42,  2,  2, 38, 42, 65, 46, 42, 15, 69, 83, 38, 54, 38,
        0, 65, 42, 55, 38, 75, 57, 42,  2, 26, 81, 21, 13, 46, 54, 38, 73,
       26, 54, 42,  2,  2, 75, 50,  8, 42,  8, 30, 16, 42, 11, 18, 42, 19,
       50,  8, 42, 35, 11,  4, 33, 16, 42, 77, 19, 39,  4, 19,  8, 16, 42,
       35, 50,  8, 30, 42, 39, 42, 85,  8, 63, 39, 77, 48, 77, 42, 33, 39,
       30, 16,  8, 16, 42, 11, 30, 42, 19, 50,  8, 42, 50, 11, 11])

In [0]:
def one_hot_encode(arr, n_labels):
    
  one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
  
  one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
  
  one_hot = one_hot.reshape((*arr.shape, n_labels))
  
  return one_hot

In [53]:
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [0]:
def get_batches(arr, batch_size, seq_length):
  batch_size_total = batch_size * seq_length
  n_batches = len(arr) // batch_size_total
  
  arr = arr[:n_batches * batch_size_total]
  arr = arr.reshape((batch_size, -1))
  
  for n in range(0, arr.shape[1], seq_length):
    x = arr[:, n:n+seq_length]
    y = np.zeros_like(x)
    try:
        y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
    except IndexError:
        y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
    yield x, y

In [0]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [56]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[46  0 26 42  2  2 38 42 65 46]
 [ 8 42 78 48 77 19 42 77 39 19]
 [19 42 19 50  8 42  8 16 63  8]
 [48 19 74 42  2  2 15 50 28  4]
 [11 16 74  7 42  2  2 38 42 44]
 [ 8 77  5 42 19 50 28 77 42 28]
 [48  4  8  5  7 42 69 39 72 50]
 [77 42 18  4 11 80  8 30 42 39]]

y
 [[ 0 26 42  2  2 38 42 65 46 42]
 [42 78 48 77 19 42 77 39 19 42]
 [42 19 50  8 42  8 16 63  8 42]
 [19 74 42  2  2 15 50 28  4 11]
 [16 74  7 42  2  2 38 42 44  8]
 [77  5 42 19 50 28 77 42 28 77]
 [ 4  8  5  7 42 69 39 72 50  8]
 [42 18  4 11 80  8 30 42 39 63]]


In [57]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class CharRNN(nn.Module):
    
  def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
    super().__init__()
    self.drop_prob = drop_prob
    self.n_layers = n_layers
    self.n_hidden = n_hidden
    self.lr = lr
    
    self.chars = tokens
    self.int2char = dict(enumerate(self.chars))
    self.char2int = {ch: ii for ii, ch in self.int2char.items()}
    
    self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, dropout=drop_prob, batch_first=True)
    
    self.dropout = nn.Dropout(drop_prob)
    
    self.fc = nn.Linear(n_hidden, len(self.chars))
    
  
  def forward(self, x, hidden):
                  
    r_output, hidden = self.lstm(x, hidden)
    
    out = self.dropout(r_output)
    
    out = out.contiguous().view(-1, self.n_hidden)
    
    out = self.fc(out)
    
    return out, hidden
  
  
  def init_hidden(self, batch_size):
      
    weight = next(self.parameters()).data
    
    if (train_on_gpu):
      hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
              weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
    else:
      hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
    
    return hidden

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    
  net.train()
  
  opt = torch.optim.Adam(net.parameters(), lr=lr)
  criterion = nn.CrossEntropyLoss()
  
  val_idx = int(len(data)*(1-val_frac))
  data, val_data = data[:val_idx], data[val_idx:]
  
  if(train_on_gpu):
    net.cuda()
  
  counter = 0
  n_chars = len(net.chars)
  for e in range(epochs):
    h = net.init_hidden(batch_size)
    
    for x, y in get_batches(data, batch_size, seq_length):
        counter += 1
        
        x = one_hot_encode(x, n_chars)
        inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
        
        if(train_on_gpu):
          inputs, targets = inputs.cuda(), targets.cuda()

        h = tuple([each.data for each in h])

        net.zero_grad()
        
        output, h = net(inputs, h)
        
        loss = criterion(output, targets.view(batch_size*seq_length).long())
        loss.backward()
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        opt.step()
        
        if counter % print_every == 0:
          val_h = net.init_hidden(batch_size)
          val_losses = []
          net.eval()
          for x, y in get_batches(val_data, batch_size, seq_length):
            x = one_hot_encode(x, n_chars)
            x, y = torch.from_numpy(x), torch.from_numpy(y)
            
            val_h = tuple([each.data for each in val_h])
            
            inputs, targets = x, y
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            output, val_h = net(inputs, val_h)
            val_loss = criterion(output, targets.view(batch_size*seq_length).long())
        
            val_losses.append(val_loss.item())
          
          net.train() 
          
          print("Epoch: {}/{}...".format(e+1, epochs),
                "Step: {}...".format(counter),
                "Loss: {:.4f}...".format(loss.item()),
                "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [60]:
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(86, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=86, bias=True)
)


In [66]:
batch_size = 128
seq_length = 100
n_epochs = 50 

train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/50... Step: 10... Loss: 1.7014... Val Loss: 1.6350
Epoch: 1/50... Step: 20... Loss: 1.6627... Val Loss: 1.6054
Epoch: 1/50... Step: 30... Loss: 1.6716... Val Loss: 1.5916
Epoch: 2/50... Step: 40... Loss: 1.6676... Val Loss: 1.5885
Epoch: 2/50... Step: 50... Loss: 1.6416... Val Loss: 1.5773
Epoch: 2/50... Step: 60... Loss: 1.6395... Val Loss: 1.5705
Epoch: 2/50... Step: 70... Loss: 1.6821... Val Loss: 1.5654
Epoch: 3/50... Step: 80... Loss: 1.6140... Val Loss: 1.5591
Epoch: 3/50... Step: 90... Loss: 1.6169... Val Loss: 1.5550
Epoch: 3/50... Step: 100... Loss: 1.6273... Val Loss: 1.5498
Epoch: 4/50... Step: 110... Loss: 1.6302... Val Loss: 1.5455
Epoch: 4/50... Step: 120... Loss: 1.5999... Val Loss: 1.5379
Epoch: 4/50... Step: 130... Loss: 1.5998... Val Loss: 1.5329
Epoch: 4/50... Step: 140... Loss: 1.6364... Val Loss: 1.5297
Epoch: 5/50... Step: 150... Loss: 1.5779... Val Loss: 1.5240
Epoch: 5/50... Step: 160... Loss: 1.5724... Val Loss: 1.5222
Epoch: 5/50... Step: 170... Loss:

In [0]:
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
  torch.save(checkpoint, f)

In [0]:
def predict(net, char, h=None, top_k=None):
    
    x = np.array([[net.char2int[char]]])
    x = one_hot_encode(x, len(net.chars))
    inputs = torch.from_numpy(x)
    
    if(train_on_gpu):
      inputs = inputs.cuda()
    
    h = tuple([each.data for each in h])
    out, h = net(inputs, h)

    p = F.softmax(out, dim=1).data
    if(train_on_gpu):
      p = p.cpu() 
    
    if top_k is None:
      top_ch = np.arange(len(net.chars))
    else:
      p, top_ch = p.topk(top_k)
      top_ch = top_ch.numpy().squeeze()
    
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p/p.sum())
    
    return net.int2char[char], h

In [0]:
def sample(net, size, prime='The', top_k=None):
        
  if(train_on_gpu):
    net.cuda()
  else:
    net.cpu()
  
  net.eval()
  
  chars = [ch for ch in prime]
  h = net.init_hidden(1)
  for ch in prime:
    char, h = predict(net, ch, h, top_k=top_k)

  chars.append(char)
  
  for ii in range(size):
    char, h = predict(net, chars[-1], h, top_k=top_k)
    chars.append(char)

  return ''.join(chars)

In [70]:
print(sample(net, 1000, prime='Percy', top_k=5))

Percy ..." 

"You must chore," Annabeth asked. "But you have a picture. You have no cabin what I was seeing thore to spend 
this wish a lift." 

She was shaken as the enemy armor was almost to bad if he was training the courtyand and the weight of 
the side of the barrier. I didn't know if what to dunt out of those leforates. 

Annabeth was still turning and carced up to the campers. 

"I've got to bat a little both." I thought he was a giant fur from my mom. 

He looked alout at the silver ground. He slapped his belating as it much, battle war shapping 
him. He was about in the widdwill and the waint doss armor than a bangera porer of terred and were straight off 




Riptide, but the starusa dreamed on a chard of a shartached. 

I dree them the staining flow, which wasn't arrow. Her scythe an armor was still buttled a stupid fold of the 
short. I chasged in the weilding in a highway, we would be back in the stitiand 
dight, as the battle blade hurt to be saving they left a throne rig

In [21]:
with open('rnn_20_epoch.net', 'rb') as f:
  checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [25]:
print(sample(loaded, 2000, top_k=5, prime="It iwas day time "))

It iwas day time and all
the prince worled him, and went out of the prievas of her
freeding. The most
change to be and
she felt
happiness, and the country to to see all at that consequence of the steps. The carriage shouted
him, he had asked his humble and couther and soft hand with the same child, that he was saying
it..

"The servom means to be makes in a long thing well all so many-came. He's the
footman's any anyweren sinceres that that would be the moment of the
sees in the water of the district persons of it. Betsine has not
come, and I'll see you
this if it such simpless of
his brother is to be more, that's to be abluctionation of the mother," he
said to him, "I
don't want to do that he saw it's such that immerse man walked it out; I can't go and see and say," he thought of his cloak with steps and with
smile.

"The children are as is a long thoughts." But there was to talk to him about it as all these whole short, with her
shoulders was so
such to hand her, and walked to the pr